In [1]:
import param as pm
import panel as pn
import secrets

from bokeh.plotting import figure
from eth_account import Account

pn.extension()

In [2]:
def generate_eth_account():
    priv = secrets.token_hex(32)
    private = "0x" + priv
    public = Account.from_key(private).address
    return (private, public)

class Wallet(pm.Parameterized):
    funds = pm.Number(100)
    def __init__(self, **params):
        super(Wallet, self).__init__(**params)
        (private, public) = generate_eth_account()
        self.private = private
        self.public = public

class Broker(Wallet):
    pass

class Owner(Wallet):
    pass

In [3]:
b = Broker()

In [4]:
b.public

'0x5B18670802B089482373C1C8439df00531e7e247'

In [5]:
b.funds

100

In [6]:
a = set()

In [7]:
a.add(1)

In [8]:
a.remove(1)

In [9]:
a

set()

In [24]:
class ProposalInverter(pm.Parameterized):
    # State
    total_funds = pm.Number(0)
    
    # Parameters
    required_stake = pm.Number(5)
    epoch_length = pm.Number(60*60*24)
    min_epochs = pm.Number(28)
    allocation_per_epoch = pm.Number(10)
    min_horizon = pm.Number(7)
    min_brokers = pm.Number(1)
    max_brokers = pm.Number(5)
    committed_brokers = set()
    broker_claimable_funds = dict()
    
    def __init__(self, owner: Wallet, initial_funds: float, **params):
        super(ProposalInverter, self).__init__(**params)
        self.owner_address = owner.public
        if owner.funds >= initial_funds:
            owner.funds -= initial_funds
            self.total_funds = initial_funds
    
    def add_broker(self, b: Broker):
        self.committed_brokers.add(b)
        
    def remove_broker(self, b: Broker):
        self.committed_brokers.remove(b)
        
    def number_of_brokers(self):
        return len(self.committed_brokers)
    
    def get_broker_claimable_funds(self):
        return self.allocation_per_epoch / self.number_of_brokers()
    
    def get_allocated_funds(self):
        pass
    
    @pm.depends("total_funds", "allocation_per_epoch", "min_horizon")
    def view(self):
        epoch = 0
        x = list()
        y = list()
        
        while self.total_funds - epoch * self.allocation_per_epoch >= self.min_horizon * self.allocation_per_epoch:
            x.append(epoch)
            y.append(self.total_funds - epoch * self.allocation_per_epoch)
            
            epoch += 1
            
        x.append(epoch)
        y.append(0)
        
        fig = figure()
        fig.line(x, y)
        
        return fig

In [25]:
patrick = Wallet(funds=2009)

In [26]:
p = ProposalInverter(owner=patrick, initial_funds=500)
pn.Row(p, p.view)

Row
    [0] Column(margin=5, name='ProposalInverter', width=300)
        [0] StaticText(value='<b>ProposalInverter</b>')
        [1] FloatInput(name='Total funds', value=500)
        [2] FloatInput(name='Required stake', value=5)
        [3] FloatInput(name='Epoch length', value=86400)
        [4] FloatInput(name='Min epochs', value=28)
        [5] FloatInput(name='Allocation per epoch', value=10)
        [6] FloatInput(name='Min horizon', value=7)
        [7] FloatInput(name='Min brokers', value=1)
        [8] FloatInput(name='Max brokers', value=5)
    [1] ParamMethod(method)

In [13]:
p.total_funds

500

In [14]:
b1 = Broker()

In [15]:
p.add_broker(b1)

In [16]:
p.committed_brokers

{Broker(funds=100, name='Broker00128')}